In [22]:
# h5py 안 될 때
#!brew reinstall hdf5
#!export CPATH="/opt/homebrew/include/"
#!export HDF5_DIR=/opt/homebrew/
#!python3 -m pip install h5py

In [23]:
# For Colab
# from google.colab import drive
# drive.mount('/content/drive')
# %cd drive/MyDrive/cuisine-prediction/Hanseul/
# !pip3 install torchmetrics

In [1]:
import os
import pickle
import math
import time
from tqdm import tqdm
from copy import deepcopy

import h5py
import numpy as np
import torch
from torch import nn, optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torch.utils.data import Dataset, Subset, DataLoader
from sklearn.metrics import f1_score

In [2]:
path_root = '../'
path_container = './Container/'

## Loading Datasets

In [3]:
with open(path_container + 'id_cuisine_dict.pickle', 'rb') as f:
    id_cuisine_dict = pickle.load(f)
with open(path_container + 'cuisine_id_dict.pickle', 'rb') as f:
    cuisine_id_dict = pickle.load(f)
with open(path_container + 'id_ingredient_dict.pickle', 'rb') as f:
    id_ingredient_dict = pickle.load(f)
with open(path_container + 'ingredient_id_dict.pickle', 'rb') as f:
    ingredient_id_dict = pickle.load(f)

In [4]:
class RecipeDataset(Dataset):
    def __init__(self, data_dir, test=False):
        self.data_dir = data_dir
        self.test = test
        with h5py.File(data_dir, 'r') as data_file:
            self.bin_data = data_file['bin_data'][:]  # Size (num_recipes=23547, num_ingredients=6714)
            if not test:
                self.int_labels = data_file['int_labels'][:]  # Size (num_recipes=23547,), about cuisines
                self.bin_labels = data_file['bin_labels'][:]  # Size (num_recipes=23547, 20), about cuisines
        
        self.padding_idx = self.bin_data.shape[1]  # == num_ingredient == 6714
        self.max_num_ingredients_per_recipe = self.bin_data.sum(1).max()  # valid & test의 경우 65
        
        # (59나 65로) 고정된 길이의 row vector에 해당 recipe의 indices 넣고 나머지는 padding index로 채워넣기
        # self.int_data: Size (num_recipes=23547, self.max_num_ingredients_per_recipe=59 or 65)
        self.int_data = np.full((len(self.bin_data), self.max_num_ingredients_per_recipe), self.padding_idx) 
        for i, bin_recipe in enumerate(self.bin_data):
            recipe = np.arange(self.padding_idx)[bin_recipe==1]
            self.int_data[i][:len(recipe)] = recipe
        
    def __len__(self):
        return len(self.bin_data)

    def __getitem__(self, idx):
        bin_data = self.bin_data[idx]
        int_data = self.int_data[idx]
        bin_label = None if self.test else self.bin_labels[idx]
        int_label = None if self.test else self.int_labels[idx]
        
        return bin_data, int_data, bin_label, int_label

In [5]:
dataset_name = ['train', 'valid_class', 'valid_compl', 'test_class', 'test_compl']

recipe_datasets = {x: RecipeDataset(os.path.join(path_container, x), test='test' in x) for x in dataset_name}

In [6]:
count_single_ingredient_recipe = 0
list_single_ingredient_recipe = []
for i in range(len(recipe_datasets['train'])):
    _bd, _,_,_ = recipe_datasets['train'][i]
    if _bd.sum()<2:
        count_single_ingredient_recipe += 1
        list_single_ingredient_recipe.append(i)
print(count_single_ingredient_recipe)
print(list_single_ingredient_recipe)

19
[564, 1263, 4074, 4203, 4901, 5277, 5360, 6232, 7835, 10585, 10777, 12476, 13301, 13989, 15951, 17374, 18153, 19039, 20469]


## Model

In [7]:
## Building blocks of Set Transformers ##
# added masks.

class MAB(nn.Module):
    def __init__(self, dim_Q, dim_K, dim_V, num_heads, ln=False, dropout=0):
        super(MAB, self).__init__()
        self.dim_V = dim_V
        self.num_heads = num_heads
        self.p = dropout
        self.fc_q = nn.Linear(dim_Q, dim_V)
        self.fc_k = nn.Linear(dim_K, dim_V)
        self.fc_v = nn.Linear(dim_K, dim_V)
        if ln:
            self.ln0 = nn.LayerNorm(dim_V)
            self.ln1 = nn.LayerNorm(dim_V)
        self.fc_o = nn.Sequential(
            nn.Linear(dim_V, dim_V),
            nn.ReLU(),
            nn.Linear(dim_V, dim_V))
        self.Dropout = nn.Dropout(p=dropout)

    def forward(self, Q, K, mask=None):
        # Q (batch, q_len, d_hid)
        # K (batch, k_len, d_hid)
        # V (batch, v_len, d_hid == dim_V)
        Q = self.fc_q(Q)
        K, V = self.fc_k(K), self.fc_v(K)
        
        dim_split = self.dim_V // self.num_heads
        
        # Q_ (batch * num_heads, q_len, d_hid // num_heads)
        # K_ (batch * num_heads, k_len, d_hid // num_heads)
        # V_ (batch * num_heads, v_len, d_hid // num_heads)
        Q_ = torch.cat(Q.split(dim_split, 2), 0)
        K_ = torch.cat(K.split(dim_split, 2), 0)
        V_ = torch.cat(V.split(dim_split, 2), 0)
        
        # energy (batch * num_heads, q_len, k_len)
        energy = Q_.bmm(K_.transpose(1,2))/math.sqrt(self.dim_V)
        if mask is not None:
            energy.masked_fill_(mask, float('-inf'))
        A = torch.softmax(energy, 2)
        
        # O (batch, q_len, d_hid)
        O = torch.cat((Q_ + A.bmm(V_)).split(Q.size(0), 0), 2)
        O = O if getattr(self, 'ln0', None) is None else self.ln0(O)
        _O = self.fc_o(O)
        if self.p > 0:
            _O = self.Dropout(_O)
        O = O + _O 
        O = O if getattr(self, 'ln1', None) is None else self.ln1(O)
        return O

class SAB(nn.Module):
    def __init__(self, dim_in, dim_out, num_heads, ln=False, dropout=0.2):
        super(SAB, self).__init__()
        self.mab = MAB(dim_in, dim_in, dim_out, num_heads, ln=ln, dropout=dropout)

    def forward(self, X, mask=None):
        return self.mab(X, X, mask=mask)

class ISAB(nn.Module):
    def __init__(self, dim_in, dim_out, num_heads, num_inds, ln=False, dropout=0.2):
        super(ISAB, self).__init__()
        self.I = nn.Parameter(torch.Tensor(1, num_inds, dim_out))
        nn.init.xavier_uniform_(self.I)
        self.mab0 = MAB(dim_out, dim_in, dim_out, num_heads, ln=ln, dropout=dropout)
        self.mab1 = MAB(dim_in, dim_out, dim_out, num_heads, ln=ln, dropout=dropout)

    def forward(self, X, mask=None):
        H = self.mab0(self.I.repeat(X.size(0), 1, 1), X, mask=mask)
        return self.mab1(X, H)

class PMA(nn.Module):
    def __init__(self, dim, num_heads, num_seeds, ln=False, dropout=0.2):
        super(PMA, self).__init__()
        self.S = nn.Parameter(torch.Tensor(1, num_seeds, dim))
        nn.init.xavier_uniform_(self.S)
        self.mab = MAB(dim, dim, dim, num_heads, ln=ln, dropout=dropout)
        
    def forward(self, X, mask=None):
        return self.mab(self.S.repeat(X.size(0), 1, 1), X, mask=mask)

In [36]:
def make_one_hot(x):
        """ Convert int_data into bin_data, if needed. """
        if type(x) is not torch.Tensor:
            x = torch.LongTensor(x)
        if x.dim() > 2:
            x = x.squeeze()
            if x.dim() > 2:
                return False
        elif x.dim() < 2:
            x = x.unsqueeze(0)
        return F.one_hot(x).sum(1)[:,:-1]

# 0416: num_output 제거. 왜냐하면 pooling layer에서 굳이 (batch 당) 두 개의 벡터를 내보낼 이유가 없다고 판단함.
# ____: 이에 따라 decoder1 제거. decoder2는 decoder로 변경. 사실 decoder 굳이 필요할까 싶어서 일단 주석처리 해둠.

class CCNet(nn.Module):
    def __init__(self, dim_embedding=256, #
                 dim_output=20,
                 num_items=6714, 
                 num_inds=32, 
                 dim_hidden=128, 
                 num_heads=4, 
                 num_enc_layers=4, 
                 num_dec_layers=2,
                 ln=True,          # LayerNorm option
                 dropout=0.2,      # Dropout option
                 classify=True,    # completion만 하고 싶으면 False로
                 complete=True,    # classification만 하고 싶으면 False로
                 freeze_classify=False, # classification만 관련된 parameter freeze
                 freeze_complete=False  # completion만 관련된 parameter freeze
                 ):
   
        super(CCNet, self).__init__()
        
        self.num_heads = num_heads
        self.padding_idx = num_items
        self.classify, self.complete = classify, complete
        self.embedding =  nn.Embedding(num_embeddings=num_items+1, embedding_dim=dim_embedding, padding_idx=-1)
        self.encoder = nn.ModuleList(
            [ISAB(dim_embedding, dim_hidden, num_heads, num_inds, ln=ln, dropout=dropout)] +
            [ISAB(dim_hidden, dim_hidden, num_heads, num_inds, ln=ln, dropout=dropout) for _ in range(num_enc_layers-1)])
        self.pooling = PMA(dim_hidden, num_heads, 1, ln=ln)
        if classify:
            self.ff1 = nn.Sequential(
                nn.BatchNorm1d(dim_hidden), nn.ReLU(), nn.Linear(dim_hidden, dim_hidden),
                nn.BatchNorm1d(dim_hidden), nn.ReLU(), nn.Linear(dim_hidden, dim_hidden))
            self.ff1_1 = nn.Sequential(
                nn.BatchNorm1d(dim_hidden), nn.ReLU(), nn.Linear(dim_hidden, dim_output))
            if freeze_classify:
                for p in self.ff1.parameters():
                    p.requires_grad = False
                for p in self.ff1_1.parameters():
                    p.requires_grad = False
        if complete:
            #self.decoder = nn.ModuleList(
            #    [MAB(dim_hidden, dim_embedding, dim_hidden, num_heads, ln=ln, dropout=dropout) for _ in range(num_dec_layers)])
            self.ff2 = nn.Sequential(
                nn.BatchNorm1d(dim_hidden), nn.ReLU(), nn.Linear(dim_hidden, dim_hidden),
                nn.BatchNorm1d(dim_hidden), nn.ReLU(), nn.Linear(dim_hidden, dim_hidden))
            self.ff2_1 = nn.Sequential(
                nn.BatchNorm1d(dim_hidden), nn.ReLU(), nn.Linear(dim_hidden, num_items))
            if freeze_complete:
                #for p in self.decoder2.parameters():
                #    p.requires_grad = False
                for p in self.ff2.parameters():
                    p.requires_grad = False
    
    def forward(self, x, bin_x=None): 
        # x(=recipes): (batch, max_num_ingredient=65) : int_data.
        if not (self.classify or self.complete):
            return
        feature = self.embedding(x)
        # feature: (batch, max_num_ingredient=65, dim_embedding=256)
        # cf. embedding.weight: (num_items+1=6715, dim_embedding=256)

        mask = (x == self.padding_idx).repeat(self.num_heads,1).unsqueeze(1)
        # mask: (batch*num_heads, 1, max_num_ingredient=65)
        
        code = feature.clone()
        for module in self.encoder:
            code = module(code, mask=mask)
        # code: (batch, max_num_ingredient=65, dim_hidden=128) : permutation-equivariant.

        signals = self.pooling(code, mask=mask)
        # signals: (batch, 1, dim_hidden=128) : permutation-invariant.

        logit_classification, logit_completion = None, None

        # Classification:
        if self.classify:
            out = self.ff1(signals.squeeze(1))  # (batch, dim_hidden)
            logit_classification = self.ff1_1(signals.squeeze(1) + out)  # residual connection, (batch, dim_output)
        
        # Completion:
        if self.complete:
            if bin_x is None:
                bin_x = make_one_hot(x)
            bool_x = (bin_x == True)

            #used_ingred_mask = bool_x.repeat(self.num_heads,1).unsqueeze(1)
            # used_ingred_mask: (batch*num_heads, 1, num_items=6714)
            
            #embedding_weight = self.embedding.weight.detach()[:-1].unsqueeze(0).repeat(feature.size(0),1,1) # no grad!
            # embedding_weight: (batch, num_items=6714, dim_embedding=256)
            
            #for module in self.decoder2:
            #    signals = module(signals, embedding_weight, mask=used_ingred_mask)
            out = self.ff2(signals.squeeze(1)) # (batch, dim_hidden)
            logit_completion = self.ff2_1(signals.squeeze(1) + out)  # residual connection, (batch, num=items=6714)
            logit_completion[bool_x] = float('-inf')

        return logit_classification, logit_completion

## Loss functions

In [37]:
class FocalLoss(nn.Module):
    """ref1: https://github.com/clcarwin/focal_loss_pytorch """
    """ref2: https://github.com/gokulprasadthekkel/pytorch-multi-class-focal-loss"""
    def __init__(self, weight=None, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, x, target):
        ce_loss = F.cross_entropy(x, target, reduction=self.reduction, weight=self.weight)
        pt = torch.exp(-ce_loss)

        loss = (1-pt)**self.gamma * ce_loss
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        return loss

In [38]:
class F1_Loss(nn.Module):
    '''
    MultiClass weighted F1_Loss. + FocalLoss.
    The original implmentation is written by Michal Haltuf on Kaggle.
    
    Reference
    ---------
    - https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
    - https://discuss.pytorch.org/t/calculating-precision-recall-and-f1-score-in-case-of-multi-label-classification/28265/6
    - http://www.ryanzhang.info/python/writing-your-own-loss-function-module-for-pytorch/
    - https://gist.github.com/SuperShinyEyes/dcc68a08ff8b615442e3bc6a9b55a354
    '''
    def __init__(self, eps=1e-8, average='macro', reduction='mean', weight=None, gamma=2):
        super().__init__()
        self.eps = eps
        self.average = average
        self.reduction = reduction
        self.weight = weight
        self.gamma = gamma
        if average not in ['macro', 'micro']:
            raise ValueError('average should be macro or micro.')
        
    def forward(self, pred, target):
        # focal loss
        ce_loss = F.cross_entropy(pred, target, reduction=self.reduction)
        pt = torch.exp(-ce_loss)

        focalloss = (1-pt)**self.gamma * ce_loss
        if self.reduction == 'mean':
            focalloss = focalloss.mean()
        elif self.reduction == 'sum':
            return focalloss.sum()
        
        # f1 loss
        target = F.one_hot(target, pred.size(-1)).float()
        pred = F.softmax(pred, dim=1)
        
        tp = (target * pred).sum(dim=0).float()
        #tn = ((1 - target) * (1 - pred)).sum(dim=0).float()
        fp = ((1 - target) * pred).sum(dim=0).float()
        fn = (target * (1 - pred)).sum(dim=0).float()

        if self.average == 'micro':
            tp, fp, fn = tp.sum(), fp.sum(), fn.sum()
        
        f1 = tp / (tp + 0.5*(fp + fn) + self.eps)
        f1.clamp(min=self.eps, max=1-self.eps)

        return 1 - f1.mean() + focalloss

## Training function

In [39]:
# WandB, 일단 뺐음
# 0415: valid loss는 classification과 completion 각각 구해서 합한 것을 취해야 할 것으로 보임.
# ____: f1_score는 sklearn.metrics.f1_score로 대체, torchmetrics의 dependency 제거, 관련 오류 고침(f1score 항상 1 나오는 오류)

def train(model,
          dataloaders,
          #criterion,
          optimizer,
          scheduler,
          #metrics,
          dataset_sizes,
          device='cpu',
          num_epochs=20,
          #wandb_log=False,
          early_stop_patience=None,
          classify=True,
          complete=True,
          random_seed=1):

    def _concatenate(running_v, new_v):
        if running_v is not None:
            return np.concatenate((running_v, new_v.clone().detach().cpu().numpy()), axis=0)
        else:
            return new_v.clone().detach().cpu().numpy()
    
    np.random.seed(random_seed)
    torch.random.manual_seed(random_seed)

    #global label_weight
    if classify:
        criterion_class = F1_Loss().to(device) #nn.CrossEntropyLoss().to(device)
    if complete:
        criterion_compl = F1_Loss().to(device) #FocalLoss(weight=None, gamma=2).to(device)

    since = time.time()

    best_model_wts = deepcopy(model.state_dict())
    best_loss = 1e4
    best_micro_f1 = 0. # classification only

    if early_stop_patience is not None:
        if not isinstance(early_stop_patience, int):
            raise TypeError('early_stop_patience should be an integer.')
        patience_cnt = 0
    
    print('-' * 5 + 'Training the model' + '-' * 5)
    for epoch in tqdm(range(num_epochs)):
        print(f'\nEpoch {epoch+1}/{num_epochs}')

        val_loss = 0. # sum of classification and completion loss

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid_class', 'valid_compl']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
                if not classify and phase == 'valid_class':
                    continue
                elif not complete and phase == 'valid_compl':
                    continue

            running_loss_class = 0.
            running_loss_compl = 0.
            running_corrects_compl = 0.
            running_corrects_class = 0.
            running_labels_class = None
            running_preds_class = None
            
            # Iterate over data.
            for idx, (bin_inputs, int_inputs, bin_labels, int_labels) in enumerate(dataloaders[phase]):
                batch_size = bin_inputs.size(0)
                if classify and phase in ['train', 'valid_class']:
                    labels_class = int_labels.to(device)
                if complete:
                    # randomly remove one ingredient for each recipe/batch
                    if phase == 'train':
                        labels_compl = torch.zeros_like(int_labels)
                        for batch in range(batch_size):
                            ingreds = torch.arange(bin_inputs.size(-1))[bin_inputs[batch]==1]
                            if len(ingreds) < 2:
                                raise RuntimeError("Train data has a single-ingredient recipe")
                            mask_ingred_idx = ingreds[np.random.randint(len(ingreds))]
                            bin_inputs[batch][mask_ingred_idx] = 0
                            int_inputs[batch][int_inputs[batch] == mask_ingred_idx] = int(bin_inputs.size(-1))
                            labels_compl[batch] = mask_ingred_idx
                        labels_compl = labels_compl.to(device)
                    elif phase == 'valid_compl':
                        labels_compl = int_labels.to(device)
                bin_inputs = bin_inputs.to(device)
                int_inputs = int_inputs.to(device)
                    
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs_class, outputs_compl = model(int_inputs, bin_x=bin_inputs)  # bin_x 없어도 작동은 가능
                    if classify and phase in ['train', 'valid_class']:
                        _, preds_class = torch.max(outputs_class, 1)
                    if complete and phase in ['train', 'valid_compl']:
                        _, preds_compl = torch.max(outputs_compl, 1)

                    if idx == 0 and phase == 'train':  # 원래 idx == 0 
                        if classify and phase in ['train', 'valid_class']:
                            print('labels_classification', labels_class.cpu().numpy())
                            #print('outputs_classification', outputs_class.clone().detach().cpu().numpy())
                            print('preds_classification', preds_class.cpu().numpy())
                        if complete and phase in ['train', 'valid_compl']:
                            print('labels_completion', labels_compl.cpu().numpy())
                            #print('outputs_completion', outputs_compl[0])
                            print('preds_completion', preds_compl.cpu().numpy())
                    
                    if classify and phase in ['train', 'valid_class']:
                        loss_class = criterion_class(outputs_class, labels_class.long())
                    if complete and phase in ['train', 'valid_compl']:
                        loss_compl = criterion_compl(outputs_compl, labels_compl.long())

                    if classify and complete and phase == 'train':
                        loss = loss_class + loss_compl
                    elif classify and phase in ['train', 'valid_class']:
                        loss = loss_class
                    elif complete and phase in ['train', 'valid_compl']:
                        loss = loss_compl

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        #torch.nn.utils.clip_grad_norm_(model.parameters(), 1) # gradient clipping
                        optimizer.step()

                if idx % 100 == 0:
                    log_str = f'    {phase} {idx * 100 // len(dataloaders[phase]):3d}% of an epoch | '
                    if classify and phase in ['train', 'valid_class']:
                        log_str += f'Loss(classif.): {loss_class.item():.4f} | '
                    if complete and phase in ['train', 'valid_compl']:
                        log_str += f'Loss(complet.): {loss_compl.item():.4f} | '
                    print(log_str)

                # statistics
                if classify and phase in ['train', 'valid_class']: # for F1 score & accuracy
                    running_loss_class += loss_class.item() * batch_size
                    running_labels_class = _concatenate(running_labels_class, labels_class)
                    running_preds_class = _concatenate(running_preds_class, preds_class)
                    running_corrects_class += torch.sum(preds_class == labels_class.data)
                if complete and phase in ['train', 'valid_compl']: # for accuracy
                    running_loss_compl += loss_compl.item() * batch_size
                    running_corrects_compl += torch.sum(preds_compl == labels_compl.data)


            epoch_loss = 0.
            log_str = f'{phase.upper()} | '
            if classify and phase in ['train', 'valid_class']:
                epoch_loss_class = running_loss_class / dataset_sizes[phase]
                epoch_loss += epoch_loss_class
                running_labels_class = torch.from_numpy(running_labels_class)
                running_preds_class = torch.from_numpy(running_preds_class)
                epoch_macro_f1 = f1_score(running_labels_class, running_preds_class, average='macro')  # classification: f1 scores.
                epoch_micro_f1 = f1_score(running_labels_class, running_preds_class, average='micro')
                epoch_acc_class = running_corrects_class / dataset_sizes[phase]
                log_str += f'Loss(classif.): {epoch_loss_class:.3f} Acc(classif.): {epoch_acc_class:.3f} Macro-F1: {epoch_macro_f1:.3f} Micro-F1: {epoch_micro_f1:.3f} | '
            if complete and phase in ['train', 'valid_compl']:
                epoch_loss_compl = running_loss_compl / dataset_sizes[phase]
                epoch_loss += epoch_loss_compl
                epoch_acc_compl = running_corrects_compl / dataset_sizes[phase]  # completion task: accuracy.
                log_str += f'Loss(complet.): {epoch_loss_compl:.3f} Acc(complet.): {epoch_acc_compl:.3f} | '
            print(log_str)
            
            if phase == 'train':
                train_loss = epoch_loss
                if classify:
                    train_macro_f1 = epoch_macro_f1
                    train_micro_f1 = epoch_micro_f1
                # if wandb_log:
                #     wandb.watch(model)
            elif 'val' in phase:
                val_loss += epoch_loss
                if classify and phase == 'valid_class':
                    val_macro_f1 = epoch_macro_f1
                    val_micro_f1 = epoch_micro_f1
            
        
        if classify and not complete:
            scheduler.step(-val_micro_f1)  # because schedular's mode == 'min'
            # deep copy the model
            if val_micro_f1 > best_micro_f1:
                best_micro_f1 = val_micro_f1
                best_loss = val_loss  # Actually, it is not the best_loss, but just save it.
                best_model_wts = deepcopy(model.state_dict())
                if early_stop_patience is not None:
                    patience_cnt = 0
            elif early_stop_patience is not None:
                patience_cnt += 1
        else:
            scheduler.step(val_loss)
            # deep copy the model
            if val_loss < best_loss:
                best_loss = val_loss
                best_model_wts = deepcopy(model.state_dict())
                if early_stop_patience is not None:
                    patience_cnt = 0
            elif early_stop_patience is not None:
                patience_cnt += 1
            

        """
        if wandb_log:
            wandb.log({'train_loss': train_loss,
                       'val_loss': val_loss,
                       'train_macro_f1': train_macro_f1,
                       'train_micro_f1': train_micro_f1,
                       'val_macro_f1': val_macro_f1,
                       'val_micro_f1': val_micro_f1,
                       'best_val_loss': best_loss,
                       'learning_rate': optimizer.param_groups[0]['lr']})
                                        # scheduler.get_last_lr()[0] for CosineAnnealingWarmRestarts
        """
        if early_stop_patience is not None:
            if patience_cnt > early_stop_patience:
                print(f'Early stop at epoch {epoch}.')
                break
        

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    print('Best val Loss: {:4f}'.format(best_loss))
    if classify and not complete:
        print('Best micro f1 score: {:4f}'.format(best_micro_f1))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_loss

## Experiment

In [40]:
def experiment(dim_embedding=256,
               dim_hidden=256,
               dropout=0.5,
               subset_length=None,
               exclude_idx=[],
               batch_size=16,
               n_epochs=50,
               lr=1e-3,
               step_size=10,  # training scheduler
               seed=0,
               classify=True,
               complete=True,
               freeze_classify=False,
               freeze_complete=False,
               pretrained_model_path=None
               ):
    
    np.random.seed(seed)
    torch.random.manual_seed(seed)
    
    dataset_name = ['train', 'valid_class', 'valid_compl']
    subset_indices = {x: [i for i in range(len(recipe_datasets[x]) if subset_length is None else subset_length)
                            if i not in exclude_idx] for x in dataset_name}
    dataloaders = {x: DataLoader(Subset(recipe_datasets[x], subset_indices[x]),
                                 batch_size=batch_size, shuffle=True) for x in dataset_name}
    dataset_sizes = {x: len(subset_indices[x]) for x in dataset_name}
    #print(dataset_sizes)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print('device: ', device)

    # Get a batch of training data
    bin_inputs, int_inputs, bin_labels, int_labels = next(iter(dataloaders['train']))
    print('inputs.shape', bin_inputs.shape, int_inputs.shape)
    print('labels.shape', bin_labels.shape, int_labels.shape)

    model_ft = CCNet(dim_embedding=dim_embedding, dim_output=len(bin_labels[0]), dim_hidden=dim_hidden,
                     num_items=len(bin_inputs[0]), num_enc_layers=4, num_dec_layers=2, ln=True, dropout=dropout,
                     classify=classify, complete=complete,
                     freeze_classify=freeze_classify, freeze_complete=freeze_complete).to(device)
    if pretrained_model_path is not None:
        pretrained_dict = torch.load(pretrained_model_path)
        model_dict = model_ft.state_dict()

        # 1. filter out unnecessary keys
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        # 2. overwrite entries in the existing state dict
        model_dict.update(pretrained_dict) 
        # 3. load the new state dict
        model.load_state_dict(pretrained_dict)
        
    print(model_ft)
    total_params = sum(dict((p.data_ptr(), p.numel()) for p in model_ft.parameters() if p.requires_grad ).values())
    print("Total Number of Parameters", total_params)

    #criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optimizer = optim.AdamW([p for p in model_ft.parameters() if p.requires_grad == True],
                                        lr=lr, betas=(0.9, 0.999), eps=1e-8, weight_decay=0.2)
    exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=step_size,
                                                      eps=1e-08, verbose=True)
    # (metric) F1Score objects - sklearn으로 대체했음. train 함수 cell 참고
    #macro_f1 = F1Score(num_classes=20, average='macro')
    #micro_f1 = F1Score(num_classes=20, average='micro')
    #metrics = {'macro_f1': macro_f1, 'micro_f1': micro_f1}

    model_ft, best_loss = train(model_ft, dataloaders, #criterion,
                                optimizer, exp_lr_scheduler, #metrics,
                                dataset_sizes, device=device, num_epochs=n_epochs, early_stop_patience=20,
                                classify=classify, complete=complete, random_seed=seed)
    
    fname = ['ckpt', 'CCNet', 'class', str(classify), 'compl', str(complete), 'best_loss', f'{best_loss:.4f}',
             'dim_embedding', str(dim_embedding), 'batch', str(batch_size), 'n_epochs', str(n_epochs),
             'lr', str(lr), 'step_size', str(step_size), 'seed', str(seed), 'subset_length', str(subset_length)]
    fname = '_'.join(fname) + '.pt'
    if not os.path.isdir('./weights/'):
        os.mkdir('./weights/')
    torch.save(model_ft.state_dict(), os.path.join('./weights/', fname))

In [43]:
# Experiment 1. classification only

# 시간: grad_clip 없을 때 기준. (gpu: 3090)
experiment(batch_size=2048, n_epochs=100, lr=1e-3, dim_embedding=256, dim_hidden=256, dropout=0.5, complete=False)

device:  cuda:0
inputs.shape torch.Size([2048, 6714]) torch.Size([2048, 59])
labels.shape torch.Size([2048, 20]) torch.Size([2048])
CCNet(
  (embedding): Embedding(6715, 256, padding_idx=6714)
  (encoder): ModuleList(
    (0): ISAB(
      (mab0): MAB(
        (fc_q): Linear(in_features=256, out_features=256, bias=True)
        (fc_k): Linear(in_features=256, out_features=256, bias=True)
        (fc_v): Linear(in_features=256, out_features=256, bias=True)
        (ln0): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ln1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (fc_o): Sequential(
          (0): Linear(in_features=256, out_features=256, bias=True)
          (1): ReLU()
          (2): Linear(in_features=256, out_features=256, bias=True)
        )
        (Dropout): Dropout(p=0.5, inplace=False)
      )
      (mab1): MAB(
        (fc_q): Linear(in_features=256, out_features=256, bias=True)
        (fc_k): Linear(in_features=256, out_features=256, bias

  0%|          | 0/100 [00:00<?, ?it/s]


Epoch 1/100
labels_classification [ 9 16  9 ...  8 12  9]
preds_classification [ 9 15  8 ... 16 18 18]
    train   0% of an epoch | Loss(classif.): 3.7268 | 
TRAIN | Loss(classif.): 2.319 Acc(classif.): 0.446 Macro-F1: 0.201 Micro-F1: 0.446 | 
    valid_class   0% of an epoch | Loss(classif.): 1.8265 | 


  1%|          | 1/100 [00:03<06:31,  3.96s/it]

VALID_CLASS | Loss(classif.): 1.826 Acc(classif.): 0.540 Macro-F1: 0.271 Micro-F1: 0.540 | 

Epoch 2/100
labels_classification [14  7  9 ...  9 16  4]
preds_classification [14  7  9 ...  9 16 11]
    train   0% of an epoch | Loss(classif.): 1.6528 | 
TRAIN | Loss(classif.): 1.534 Acc(classif.): 0.589 Macro-F1: 0.356 Micro-F1: 0.589 | 
    valid_class   0% of an epoch | Loss(classif.): 1.4914 | 


...

VALID_CLASS | Loss(classif.): 1.626 Acc(classif.): 0.697 Macro-F1: 0.603 Micro-F1: 0.697 | 

Epoch 54/100
labels_classification [ 9  3  7 ... 17  9 12]
preds_classification [ 9  3  7 ... 17  9 12]
    train   0% of an epoch | Loss(classif.): 0.0425 | 
TRAIN | Loss(classif.): 0.040 Acc(classif.): 0.972 Macro-F1: 0.969 Micro-F1: 0.972 | 
    valid_class   0% of an epoch | Loss(classif.): 1.5683 | 


 54%|█████▍    | 54/100 [03:32<03:00,  3.93s/it]

VALID_CLASS | Loss(classif.): 1.626 Acc(classif.): 0.697 Macro-F1: 0.603 Micro-F1: 0.697 | 

Epoch 55/100
labels_classification [ 7 12  1 ... 18  7 14]
preds_classification [ 7 12  1 ... 18  7 14]
    train   0% of an epoch | Loss(classif.): 0.0445 | 
TRAIN | Loss(classif.): 0.040 Acc(classif.): 0.973 Macro-F1: 0.969 Micro-F1: 0.973 | 
    valid_class   0% of an epoch | Loss(classif.): 1.5735 | 


 54%|█████▍    | 54/100 [03:36<03:04,  4.01s/it]

VALID_CLASS | Loss(classif.): 1.624 Acc(classif.): 0.697 Macro-F1: 0.604 Micro-F1: 0.697 | 
Early stop at epoch 54.
Training complete in 3m 37s
Best val Loss: 1.558614
Best micro f1 score: 0.702982


In [44]:
experiment(batch_size=1024, n_epochs=100, lr=1e-3, dim_embedding=256, dim_hidden=256, dropout=0.5, complete=False)

device:  cuda:0
inputs.shape torch.Size([1024, 6714]) torch.Size([1024, 59])
labels.shape torch.Size([1024, 20]) torch.Size([1024])
CCNet(
  (embedding): Embedding(6715, 256, padding_idx=6714)
  (encoder): ModuleList(
    (0): ISAB(
      (mab0): MAB(
        (fc_q): Linear(in_features=256, out_features=256, bias=True)
        (fc_k): Linear(in_features=256, out_features=256, bias=True)
        (fc_v): Linear(in_features=256, out_features=256, bias=True)
        (ln0): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ln1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (fc_o): Sequential(
          (0): Linear(in_features=256, out_features=256, bias=True)
          (1): ReLU()
          (2): Linear(in_features=256, out_features=256, bias=True)
        )
        (Dropout): Dropout(p=0.5, inplace=False)
      )
      (mab1): MAB(
        (fc_q): Linear(in_features=256, out_features=256, bias=True)
        (fc_k): Linear(in_features=256, out_features=256, bias

  0%|          | 0/100 [00:00<?, ?it/s]


Epoch 1/100
labels_classification [ 9 16  9 ... 13 13  2]
preds_classification [18 15  8 ... 18 12  1]
    train   0% of an epoch | Loss(classif.): 3.7346 | 
TRAIN | Loss(classif.): 2.062 Acc(classif.): 0.488 Macro-F1: 0.246 Micro-F1: 0.488 | 
    valid_class   0% of an epoch | Loss(classif.): 1.6926 | 


  1%|          | 1/100 [00:04<07:08,  4.33s/it]

VALID_CLASS | Loss(classif.): 1.668 Acc(classif.): 0.560 Macro-F1: 0.301 Micro-F1: 0.560 | 

Epoch 2/100
labels_classification [14  7  9 ... 19  7 19]
preds_classification [14  7  9 ... 18  7 19]
    train   0% of an epoch | Loss(classif.): 1.4687 | 
TRAIN | Loss(classif.): 1.360 Acc(classif.): 0.617 Macro-F1: 0.415 Micro-F1: 0.617 | 
    valid_class   0% of an epoch | Loss(classif.): 1.3724 | 


...

VALID_CLASS | Loss(classif.): 1.759 Acc(classif.): 0.708 Macro-F1: 0.619 Micro-F1: 0.708 | 

Epoch 82/100
labels_classification [ 9  9 12 ...  2  9  9]
preds_classification [ 9  9 12 ...  2  9  9]
    train   0% of an epoch | Loss(classif.): 0.0191 | 
TRAIN | Loss(classif.): 0.024 Acc(classif.): 0.982 Macro-F1: 0.980 Micro-F1: 0.982 | 
    valid_class   0% of an epoch | Loss(classif.): 1.6879 | 


 82%|████████▏ | 82/100 [05:51<01:16,  4.26s/it]

VALID_CLASS | Loss(classif.): 1.754 Acc(classif.): 0.708 Macro-F1: 0.620 Micro-F1: 0.708 | 

Epoch 83/100
labels_classification [ 5  2 16 ...  9  1  0]
preds_classification [ 5  2 16 ...  9  1  0]
    train   0% of an epoch | Loss(classif.): 0.0192 | 
TRAIN | Loss(classif.): 0.024 Acc(classif.): 0.982 Macro-F1: 0.980 Micro-F1: 0.982 | 
    valid_class   0% of an epoch | Loss(classif.): 1.8859 | 


 82%|████████▏ | 82/100 [05:55<01:18,  4.34s/it]

VALID_CLASS | Loss(classif.): 1.747 Acc(classif.): 0.709 Macro-F1: 0.620 Micro-F1: 0.709 | 
Early stop at epoch 82.
Training complete in 5m 56s
Best val Loss: 1.730756
Best micro f1 score: 0.711137


In [45]:
experiment(batch_size=512, n_epochs=100, lr=1e-3, dim_embedding=256, dim_hidden=256, dropout=0.5, complete=False)

device:  cuda:0
inputs.shape torch.Size([512, 6714]) torch.Size([512, 59])
labels.shape torch.Size([512, 20]) torch.Size([512])
CCNet(
  (embedding): Embedding(6715, 256, padding_idx=6714)
  (encoder): ModuleList(
    (0): ISAB(
      (mab0): MAB(
        (fc_q): Linear(in_features=256, out_features=256, bias=True)
        (fc_k): Linear(in_features=256, out_features=256, bias=True)
        (fc_v): Linear(in_features=256, out_features=256, bias=True)
        (ln0): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ln1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (fc_o): Sequential(
          (0): Linear(in_features=256, out_features=256, bias=True)
          (1): ReLU()
          (2): Linear(in_features=256, out_features=256, bias=True)
        )
        (Dropout): Dropout(p=0.5, inplace=False)
      )
      (mab1): MAB(
        (fc_q): Linear(in_features=256, out_features=256, bias=True)
        (fc_k): Linear(in_features=256, out_features=256, bias=Tru

  0%|          | 0/100 [00:00<?, ?it/s]


Epoch 1/100
labels_classification [ 9 16  9  5 13 11  9  3  9 16 11 16  3 18  9 14  7 13 11 16 13 13  5 16
  3 13  9  7 13  5  5  5  3  5 10 18 13  9 12  3 17 16  7  9  9 16 13 13
  2 13  5  8  9 14 16 12 12  7 13  7  0 14  9  5 13  2  5  9  7 12 13  5
  8 13 13  1  9  5 16  9  4 17 11  2 14  3  5  7  9  3 18 13 18  3  2 13
  7  9  7 13 16  1  6  9 16 13  6  7  9  4 11  2 13  5  5 16 13 18 13  9
 17 11  9 12  3 14  8  3  7  3  9  6 17 13 10  9  9  7  5  9  8 13 16 11
  8  9  7  9  4 16 13  3 13  2  7 13 16  3  5 16 19 12 16  6  9 13  9 16
 13 13  9  4  6 16  3 11  9 11  5  0 13  8  9  7  4  3  5 13  7 13  8  3
  5 16 16 16 16  7 16  0  5  2  3  5  5 16  6  7 19 13 16 16  9 16  3  3
  9  4 13 14  0 14  9 19 13 16  5 13  5  7 16  8  7 13  8  3  1  2  3 18
  9  9  9  9 11  9 12 18 13 16  5  2  9 16 18  3  5  9 16 11  5 19 18  0
  2 16 10  9 18  7 11  1  7 18 14  9  8 13  4  5  9  6  7  9  5  9 13  2
 16 16  9 15  5  9  6 18 11  5  5 13  9  3  5 10 18 16  3  5 19  9 13  3
  9 17  7  7 18 

  1%|          | 1/100 [00:04<07:42,  4.67s/it]

VALID_CLASS | Loss(classif.): 1.540 Acc(classif.): 0.584 Macro-F1: 0.349 Micro-F1: 0.584 | 

Epoch 2/100
labels_classification [14  7  9 13  9  8  9  9  3  8  3 10 11 13  9  9 13 19 13 16  7 13 16 13
 17  7 13 17  0 16 17  5  5 16  9 13  1  9  5  7  5  7  7  9  0 16  2  6
 13  9 13  8 13 13 16 13 15  9 16  6  9  9  9 18 17 11 17 12  3 13 19  2
  3 13 11  9  9  3  3 12  7  9  9 17  4 11 13  7  5 16  7 13  9 16  3 16
 19  2  0 13 15 16  2 12 13 13  7  2  9 14 13  2  1  2 19  6  3 18 17  9
  9  6 16  6  0 18  2 16 19  5  5  3  6  0 11  9  9  3 17 11  3 16  5  3
 13 16  7  3  7 19  9 16 13 13  7  9  3  2  5  5  9  6 19  8 18  9 13 13
 16  9  9 16  9  9 16  6  1  9  7 12  9  7  9 18  5  7  5 13  7 18  4  1
 13  9 16 18  3 13  9  1  7 16  6 13 16  1  9 13 13 18 13 16  7  7  2  9
 18  3  3  9  5  2 13  7 12 16  9 13  1  4  5 18  9  5  9 13  3 16 16 12
  2  9 16  3  5 13 19  9  4  6 17  4 17 13  9 13 13  9  9  9  9  4 13 13
  9  9  9 10 18  5 16  3 14  3  9 13  9 17 11 11 12  1  9  6  5 16 16 

...

VALID_CLASS | Loss(classif.): 1.454 Acc(classif.): 0.721 Macro-F1: 0.628 Micro-F1: 0.721 | 

Epoch 71/100
labels_classification [ 3  5 13 16  6 13  9  9  9  9 16 16  9  9 18 13 11 16  9  9  9  9  1  4
 13  9 13  9 16  3 11 18 11 16 16  6  4  1 15  3  9  9 16 18  9  3  3 13
  7 17 16  0  9  9 13 16 11  5  8  7 13  6  6  9  5  8  3 11 13 18 17  9
 16  9 13  9  5 16  9 13  9 17  6 16  7 18  9 15 18  9  7 18  9 16  5 13
  9  2  7  7 13  2 13 18  6 11 16  5  9 16 13 13 12 14 13  9  7 13  4  9
  9 17 12 16 13 16  9 19  7 13  5  7 11 13 13  9  9 16 13  9 13 18  7 13
 16 19 10  9  1  9  3  9 18 16  9  9  3  3  9 16  2 13  3 16 14 13  0  8
  5  6  0 14 12 11 16  1  7  3  9 13 16  9 13  9 13 16 13 13 16  9 17  3
  3  9 10  2  1  7  4  9 13  9  3 16  0 16  9 16  9  8 16 16  9  9 13  1
  7 13  0  6  3 16  7  0  3 13 12 18  9 13 13  9 16  7 16  3  9 16 19  9
 11  9  5  9  2 16  9 13 16  1 12  7 16  3  5  2 14  9 14 13 17  9  5  2
 19  9  9 18  3  2 16  6  8  7  3 17 18 13  3  3  5 18 18 13  6  3 13

 71%|███████   | 71/100 [05:36<02:16,  4.72s/it]

VALID_CLASS | Loss(classif.): 1.460 Acc(classif.): 0.721 Macro-F1: 0.628 Micro-F1: 0.721 | 

Epoch 72/100
labels_classification [ 3 13  7 19  0 13 13  9 18 18  8 16  3 16 14 16 15  9  9 19  4  4 13  4
 13  9  8 16  3  9 16  9  9  6 16  9 18  4 11  4  7  9  6  2 16  9 13  5
 18 13  9  9  9  9 13  9 16  3 13  3  2 12  9  0  4  9  3  9 13  3 14 16
 16 13  1  4 13 13  5  5 13 16  3  9 16 18  9 14 13 13 13 15  9 18 16  9
  3  9  9  5  6 11 16 19 19 13  9 13 18 19 11 13  2  9 16 13 13 19 16  9
  3 14 13  2 16 19  9  1 11  9  9  7  9  9 13  9 13 13  2 16  3  1 13  9
  2  4 16  9 13 13 13 10  9  7  1 13 14  0  1  7 13 17  9  4  3 18 13  9
 16 14 13 11 13 16  3  9 14  6 12  9  5  9  6  4 12 12 16  6  3 14  9  3
  3  4  6 12  9 13  9 11  3  5 11  9  3 16 13 10  9 16  7 13 18  9  2  5
 14 13 10  2 19  9 14  2 13 13  7 17  9 18 11  5  9  9 16 17  4  3 15  9
  9 13  3 16 19 19  7 13  6 13  7 11  2  5 13  9 11 18  8 16  4 13  7  5
 14  3 10 13 13  9  9  9 16 18  3  9 13  3  5 13  7 16  2 13 18  2  6

 71%|███████   | 71/100 [05:40<02:19,  4.80s/it]

VALID_CLASS | Loss(classif.): 1.466 Acc(classif.): 0.720 Macro-F1: 0.627 Micro-F1: 0.720 | 
Early stop at epoch 71.
Training complete in 5m 41s
Best val Loss: 1.403080
Best micro f1 score: 0.724388


In [46]:
experiment(batch_size=256, n_epochs=100, lr=1e-3, dim_embedding=256, dim_hidden=256, dropout=0.5, complete=False)

device:  cuda:0
inputs.shape torch.Size([256, 6714]) torch.Size([256, 59])
labels.shape torch.Size([256, 20]) torch.Size([256])
CCNet(
  (embedding): Embedding(6715, 256, padding_idx=6714)
  (encoder): ModuleList(
    (0): ISAB(
      (mab0): MAB(
        (fc_q): Linear(in_features=256, out_features=256, bias=True)
        (fc_k): Linear(in_features=256, out_features=256, bias=True)
        (fc_v): Linear(in_features=256, out_features=256, bias=True)
        (ln0): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ln1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (fc_o): Sequential(
          (0): Linear(in_features=256, out_features=256, bias=True)
          (1): ReLU()
          (2): Linear(in_features=256, out_features=256, bias=True)
        )
        (Dropout): Dropout(p=0.5, inplace=False)
      )
      (mab1): MAB(
        (fc_q): Linear(in_features=256, out_features=256, bias=True)
        (fc_k): Linear(in_features=256, out_features=256, bias=Tru

  0%|          | 0/100 [00:00<?, ?it/s]


Epoch 1/100
labels_classification [ 9 16  9  5 13 11  9  3  9 16 11 16  3 18  9 14  7 13 11 16 13 13  5 16
  3 13  9  7 13  5  5  5  3  5 10 18 13  9 12  3 17 16  7  9  9 16 13 13
  2 13  5  8  9 14 16 12 12  7 13  7  0 14  9  5 13  2  5  9  7 12 13  5
  8 13 13  1  9  5 16  9  4 17 11  2 14  3  5  7  9  3 18 13 18  3  2 13
  7  9  7 13 16  1  6  9 16 13  6  7  9  4 11  2 13  5  5 16 13 18 13  9
 17 11  9 12  3 14  8  3  7  3  9  6 17 13 10  9  9  7  5  9  8 13 16 11
  8  9  7  9  4 16 13  3 13  2  7 13 16  3  5 16 19 12 16  6  9 13  9 16
 13 13  9  4  6 16  3 11  9 11  5  0 13  8  9  7  4  3  5 13  7 13  8  3
  5 16 16 16 16  7 16  0  5  2  3  5  5 16  6  7 19 13 16 16  9 16  3  3
  9  4 13 14  0 14  9 19 13 16  5 13  5  7 16  8  7 13  8  3  1  2  3 18
  9  9  9  9 11  9 12 18 13 16  5  2  9 16 18  3]
preds_classification [ 9 15  8 18  9 12 18  2 12  0 16  7 11 18  8 18  1  7  5  1  7 18 16  2
  1 18 17  4 11  1  8 15 18  1 11  4  1  4  1 11  4  9 11  1 12  7  9  1
  9  1 12 17 16 15

  1%|          | 1/100 [00:05<09:03,  5.49s/it]

VALID_CLASS | Loss(classif.): 1.624 Acc(classif.): 0.563 Macro-F1: 0.346 Micro-F1: 0.563 | 

Epoch 2/100
labels_classification [14  7  9 13  9  8  9  9  3  8  3 10 11 13  9  9 13 19 13 16  7 13 16 13
 17  7 13 17  0 16 17  5  5 16  9 13  1  9  5  7  5  7  7  9  0 16  2  6
 13  9 13  8 13 13 16 13 15  9 16  6  9  9  9 18 17 11 17 12  3 13 19  2
  3 13 11  9  9  3  3 12  7  9  9 17  4 11 13  7  5 16  7 13  9 16  3 16
 19  2  0 13 15 16  2 12 13 13  7  2  9 14 13  2  1  2 19  6  3 18 17  9
  9  6 16  6  0 18  2 16 19  5  5  3  6  0 11  9  9  3 17 11  3 16  5  3
 13 16  7  3  7 19  9 16 13 13  7  9  3  2  5  5  9  6 19  8 18  9 13 13
 16  9  9 16  9  9 16  6  1  9  7 12  9  7  9 18  5  7  5 13  7 18  4  1
 13  9 16 18  3 13  9  1  7 16  6 13 16  1  9 13 13 18 13 16  7  7  2  9
 18  3  3  9  5  2 13  7 12 16  9 13  1  4  5 18  9  5  9 13  3 16 16 12
  2  9 16  3  5 13 19  9  4  6 17  4 17 13  9 13]
preds_classification [14  7  9 13  9  9  9  9  3  5 12 18  3 13  9  9 13 13 13  2  7 13 16 13

...

VALID_CLASS | Loss(classif.): 1.444 Acc(classif.): 0.725 Macro-F1: 0.643 Micro-F1: 0.725 | 

Epoch 65/100
labels_classification [16  7 17  5  2 18 13 13 10  9  2  4  1  7  8  7  0 16  7 13 14  9  2  6
 13  5  9  9  7  3 16 15  5 17  7  6 13 16  5  9  2  3 16  9  0  9  9 13
  9  2  5 16  5  7  5  9 11 11  9 13 14 16 13  9 16  7 13  8  9  5 10 13
 11 16 16  2 16 13  6 16 16  9 17  9  2 13 13 16  1  9  1  3  3  1 14 16
  9  5 13  4  9  9  5 13  5  4 13  7  5 17 17 10  2  9  8 13  5 17  5  1
 16  2  5 16  7 13  2  7  9  1 13 19 13 13 16 13  9 13  2 13  2 13 13 13
  0  5  0 19 19  9  9  9  9 13  5 16 15  6  9 19  9  7  7  9 11 18 14  9
 19  3  9 18  9 17 16  4 13  7 18  9  5 13  7  5 16  9  9  5  7  4 16  9
 13 18 11  7  7 18  2  2  9 13 16  7  2  9  3 13 11 13 17 13  7  5  3  5
 15 13 18  1  9 18  9 13  7 16  9  9 13  5  9  1 16 13  9  9  5  5 13 13
  9  5 13  5 16 19  3 18 13  3  9 16  5  3  3 11]
preds_classification [16  7 17  5  2 18 13 13 10  9  2  4  1  7  8  7  0 16  7 13 14  9  2  

 65%|██████▌   | 65/100 [05:53<03:09,  5.41s/it]

VALID_CLASS | Loss(classif.): 1.446 Acc(classif.): 0.724 Macro-F1: 0.643 Micro-F1: 0.724 | 

Epoch 66/100
labels_classification [12  7 16 13 11 11  7  9  6  2 16  7  7 19  3  9 13  5 13 16 13  9 16 16
  3 13  5 13 13 12 13 13  0  9 18 17  9  9 13 12  3 13  7 13  7  7  5 13
 16 15 17  4  7 19  7 18  9  7 13  9  1  9  5 18 16 16 12  9 16  9  7 14
 13  5 16 16  5  9  1  9 10  9 16  5  7  9 13  9  9  8  5  5  4  9 11  3
 13 13 12  9 16 15  7 14 17 12  9  9  2  5 14 16  6  1 12 18  7  5 14 13
  9 16 14 13 16  3  0  9  9 16  7  6  2  9  4  2 11  5  4 16 13 16  3  9
  7 12 11 16  3  4 18 13 15  9 13 17  1 16  4 17 13 13  9  7 18  2 13 13
 13  7  9  7  9 13 13  5 11  5  1 16 13  1  9 11 13 13 13 16 18  5 16 13
  7  9  9 13 15 13 11  1 16  9  9  9 16 16 13  2  9  1  3  5  5  2 16 13
  5  3  3 16 11 13  5 11  5  9 13 16  7  5 13 19 13 18  9 13  3 13  6  2
 11  9 13 13  7 16 13  7 13  6 16  7  5  2  3  9]
preds_classification [12  7 16 13 11 11  7  9  6  2 16  7  7 19  3  9 13  5 13 16 13  9 16 1

 65%|██████▌   | 65/100 [05:58<03:13,  5.52s/it]

VALID_CLASS | Loss(classif.): 1.461 Acc(classif.): 0.725 Macro-F1: 0.645 Micro-F1: 0.725 | 
Early stop at epoch 65.
Training complete in 5m 59s
Best val Loss: 1.370409
Best micro f1 score: 0.729358


In [47]:
experiment(batch_size=128, n_epochs=100, lr=1e-3, dim_embedding=256, dim_hidden=256, dropout=0.5, complete=False)  # (Macro f1 0.658)

device:  cuda:0
inputs.shape torch.Size([128, 6714]) torch.Size([128, 59])
labels.shape torch.Size([128, 20]) torch.Size([128])
CCNet(
  (embedding): Embedding(6715, 256, padding_idx=6714)
  (encoder): ModuleList(
    (0): ISAB(
      (mab0): MAB(
        (fc_q): Linear(in_features=256, out_features=256, bias=True)
        (fc_k): Linear(in_features=256, out_features=256, bias=True)
        (fc_v): Linear(in_features=256, out_features=256, bias=True)
        (ln0): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ln1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (fc_o): Sequential(
          (0): Linear(in_features=256, out_features=256, bias=True)
          (1): ReLU()
          (2): Linear(in_features=256, out_features=256, bias=True)
        )
        (Dropout): Dropout(p=0.5, inplace=False)
      )
      (mab1): MAB(
        (fc_q): Linear(in_features=256, out_features=256, bias=True)
        (fc_k): Linear(in_features=256, out_features=256, bias=Tru

  0%|          | 0/100 [00:00<?, ?it/s]


Epoch 1/100
labels_classification [ 9 16  9  5 13 11  9  3  9 16 11 16  3 18  9 14  7 13 11 16 13 13  5 16
  3 13  9  7 13  5  5  5  3  5 10 18 13  9 12  3 17 16  7  9  9 16 13 13
  2 13  5  8  9 14 16 12 12  7 13  7  0 14  9  5 13  2  5  9  7 12 13  5
  8 13 13  1  9  5 16  9  4 17 11  2 14  3  5  7  9  3 18 13 18  3  2 13
  7  9  7 13 16  1  6  9 16 13  6  7  9  4 11  2 13  5  5 16 13 18 13  9
 17 11  9 12  3 14  8  3]
preds_classification [18 15  8 18  9 12 18 18  7 18  9  7 11 18 11 18  1  7  2  1  7 18 16  2
  1 18 17  1 11 18 12  2 18 18 11  4  1  4  1  9  4 17 11  1 12  7  9  1
  9  1 18 12 16  1  2 12  1  9 15  1 16 18  1  9 12  2  9 12  9 18 15  7
  9  5 18  2 18  7 18  4 18  2 12 11 12  1 12 14  7 15  2  1  1  1  9  7
 18  9  2 18  9 12  9  1 11  2  1 11  9 16  9 19 11 11  1  9 12 11 11 12
 15  9  7 12 18  1 18  1]
    train   0% of an epoch | Loss(classif.): 3.7590 | 
    train  54% of an epoch | Loss(classif.): 1.7627 | 
TRAIN | Loss(classif.): 1.805 Acc(classif.): 0.532 M

  1%|          | 1/100 [00:07<12:22,  7.50s/it]

VALID_CLASS | Loss(classif.): 1.697 Acc(classif.): 0.542 Macro-F1: 0.346 Micro-F1: 0.542 | 

Epoch 2/100
labels_classification [14  7  9 13  9  8  9  9  3  8  3 10 11 13  9  9 13 19 13 16  7 13 16 13
 17  7 13 17  0 16 17  5  5 16  9 13  1  9  5  7  5  7  7  9  0 16  2  6
 13  9 13  8 13 13 16 13 15  9 16  6  9  9  9 18 17 11 17 12  3 13 19  2
  3 13 11  9  9  3  3 12  7  9  9 17  4 11 13  7  5 16  7 13  9 16  3 16
 19  2  0 13 15 16  2 12 13 13  7  2  9 14 13  2  1  2 19  6  3 18 17  9
  9  6 16  6  0 18  2 16]
preds_classification [14 14  9 13  9  5  9  9  3  5  3 18  3 13  9  9 13  3 13 16  7 13 16 13
  9  7 13  9  6 16  9  5 16 16  9 13  5  5  5  7 16  7  7  9 13  5  0  6
 13  9 13 16 13 13 16 13 16  2 13  9  9  9  9 18 13 11 13  3  3 13 19 16
  3 13 11  9  9  3  3  3  7  9  5  6 18  3  6  7  9 16  7 13  9  0  3 16
 16  2 13 13 11  2  2  3 13 13  7  2  9 14 13  2  5  9 18  6  3 18  5  9
  9  9 16  9 13 18  2 16]
    train   0% of an epoch | Loss(classif.): 1.2073 | 
    train  54% 

...

VALID_CLASS | Loss(classif.): 1.349 Acc(classif.): 0.733 Macro-F1: 0.646 Micro-F1: 0.733 | 

Epoch 51/100
labels_classification [ 9  7 11  2 13 16  9  2 18 16 16  9 14  9 13  6 13  3  7 13 13  5  1 13
  3  7 13  3  1 18 13  9  5  5 18  9  9  9  9  1 16  3  9  9  5  4 12  8
  9 14 11 16  9  9  7 19  5 16  0  7 13 13  1  9 16  9 13  9  6 15 10 11
 17 17 18  7  3  9 13  8  6  5 13  7  7  9  5 18 13  1 16 16  3  9 19  3
  9  7  5  5 13 15  7 16 13  4 16 13  4  2  5  9 13  8 13  2  7 13 13  9
  5  9  9  9 12  5 13  7]
preds_classification [ 9  7 11  2 13 16  9  2 18 16 16  9 14  9 13  6 13  3  7 13 13  9  1 13
  3  7 13  3  1 18 13  9  5  5 18  9  9  9  9  1 16  3  9  9  5  4 12  8
  9 14 11 16  9  9  7 19  9 16  0  7 13 13  1  9 16  9 13  9  6 15 10 11
 17 17 18  7  3  9 13  8  6  5 13  7  7  9  5 18 13  1 16 16  3  9 19  3
  9  7  5  5 13 15  7 16 13  4 16 13  4  2  5  9 13  8 13  2  7 13 13 13
  5  9  9  9 12  5 13  7]
    train   0% of an epoch | Loss(classif.): 0.0152 | 
    train  54%

 51%|█████     | 51/100 [06:27<06:12,  7.60s/it]

VALID_CLASS | Loss(classif.): 1.355 Acc(classif.): 0.734 Macro-F1: 0.646 Micro-F1: 0.734 | 

Epoch 52/100
labels_classification [18  9  9 13  7  9 16 16 16  9  3 11  9  9 13  5 13  5 17  9 14  5 13  9
  9 13 13 10  7 13 16 13 13 14  8  7  2 16  3  2 19  2 16 15  9 13 14 13
  3  7 10  5  9 13  5 13 13 17  9  9  3  9 18  9 13  5  9 16  9 18 13  2
  5  5  2  3 16  7  9  6  7 15  2 13 16  8  8  9  9  9  2  9 15  9 13 11
 18 18  5  9 13 13 16  3  9  8 16  9 13  5 11 13  9  2  7 13  9 13  9 13
 16  9  0 13 11  1  9 13]
preds_classification [18  9  9 13  7  9 16 16 16  9  3 11  9  9 13  5 13  5 17  9 14  5 13  9
  9 13 13 10  7 13 16 13 13 14 16  7  2 16  3  2 19  2 16 15  9 13 14 13
  3  7 10  5  9 13  5 13 13 17  9  9  3  9 18  9 13  5  9 16  9 18 13  2
  5  5  2  3 16  7  9  6  7 15  2 13 16  8  8  9  9  9  2  9 15  9 13 11
 18 18  5  9 13 13 16  3  9  8 16  9 13  5 11 13  9  2  7 13  9 13  9 13
 16  9  0 13 11  1  9 13]
    train   0% of an epoch | Loss(classif.): 0.1127 | 
    train  54%

 51%|█████     | 51/100 [06:34<06:19,  7.74s/it]

VALID_CLASS | Loss(classif.): 1.360 Acc(classif.): 0.734 Macro-F1: 0.647 Micro-F1: 0.734 | 
Early stop at epoch 51.
Training complete in 6m 35s
Best val Loss: 1.203273
Best micro f1 score: 0.740443


In [48]:
experiment(batch_size=64, n_epochs=100, lr=1e-3, dim_embedding=256, dim_hidden=256, dropout=0.5, complete=False)  # best micro f1 score (Macro f1 0.661)

device:  cuda:0
inputs.shape torch.Size([64, 6714]) torch.Size([64, 59])
labels.shape torch.Size([64, 20]) torch.Size([64])
CCNet(
  (embedding): Embedding(6715, 256, padding_idx=6714)
  (encoder): ModuleList(
    (0): ISAB(
      (mab0): MAB(
        (fc_q): Linear(in_features=256, out_features=256, bias=True)
        (fc_k): Linear(in_features=256, out_features=256, bias=True)
        (fc_v): Linear(in_features=256, out_features=256, bias=True)
        (ln0): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ln1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (fc_o): Sequential(
          (0): Linear(in_features=256, out_features=256, bias=True)
          (1): ReLU()
          (2): Linear(in_features=256, out_features=256, bias=True)
        )
        (Dropout): Dropout(p=0.5, inplace=False)
      )
      (mab1): MAB(
        (fc_q): Linear(in_features=256, out_features=256, bias=True)
        (fc_k): Linear(in_features=256, out_features=256, bias=True)
 

  0%|          | 0/100 [00:00<?, ?it/s]


Epoch 1/100
labels_classification [ 9 16  9  5 13 11  9  3  9 16 11 16  3 18  9 14  7 13 11 16 13 13  5 16
  3 13  9  7 13  5  5  5  3  5 10 18 13  9 12  3 17 16  7  9  9 16 13 13
  2 13  5  8  9 14 16 12 12  7 13  7  0 14  9  5]
preds_classification [18 15  8  7  9 12 18  7 12  1 16  7 11 18 11 18  1  7  5  1  7 18 16  2
  1 18 15  1 11  1  8  2  1 18 11  4  1  7  1 11  4 12 11  1 11  7  9  1
  9  1 12 12 16 15  2 12  1  9 15  1  7 18  1  9]
    train   0% of an epoch | Loss(classif.): 3.7303 | 
    train  27% of an epoch | Loss(classif.): 2.2608 | 
    train  54% of an epoch | Loss(classif.): 1.6192 | 
    train  81% of an epoch | Loss(classif.): 1.4857 | 
TRAIN | Loss(classif.): 1.859 Acc(classif.): 0.525 Macro-F1: 0.275 Micro-F1: 0.525 | 
    valid_class   0% of an epoch | Loss(classif.): 1.8421 | 
    valid_class  81% of an epoch | Loss(classif.): 1.5148 | 


  1%|          | 1/100 [00:13<21:43, 13.17s/it]

VALID_CLASS | Loss(classif.): 1.601 Acc(classif.): 0.582 Macro-F1: 0.337 Micro-F1: 0.582 | 

Epoch 2/100
labels_classification [14  7  9 13  9  8  9  9  3  8  3 10 11 13  9  9 13 19 13 16  7 13 16 13
 17  7 13 17  0 16 17  5  5 16  9 13  1  9  5  7  5  7  7  9  0 16  2  6
 13  9 13  8 13 13 16 13 15  9 16  6  9  9  9 18]
preds_classification [14  7  9 13  9  9  9  9 12  9 12 18  3 13  9  9 13  3 13 16  7 13 16 13
  9  7 13  9  9 16  5  9  5 16  9 13  5  5  5  7 16  7  7  9 16  5  2  9
 13  9 13  9 13 13 16 13  5 16 16  9  9  9  9 18]
    train   0% of an epoch | Loss(classif.): 1.2157 | 
    train  27% of an epoch | Loss(classif.): 1.1443 | 
    train  54% of an epoch | Loss(classif.): 1.0367 | 
    train  81% of an epoch | Loss(classif.): 1.2180 | 
TRAIN | Loss(classif.): 1.279 Acc(classif.): 0.645 Macro-F1: 0.444 Micro-F1: 0.645 | 
    valid_class   0% of an epoch | Loss(classif.): 1.0006 | 
    valid_class  81% of an epoch | Loss(classif.): 1.5292 | 


...

VALID_CLASS | Loss(classif.): 1.625 Acc(classif.): 0.736 Macro-F1: 0.651 Micro-F1: 0.736 | 

Epoch 49/100
labels_classification [ 4  7  9  9  9  9  5  2 11 11  2 13  4  3  6  2 17 11 13 13  5  3 16  7
  5 13  9  7  2 13  7 16 13  0 13  5 18  2  3  9  9  3  6  7  9  5  9 13
  7  7 13 16 16  9 12  7  5 13  7  3 13  9 13 19]
preds_classification [ 4  7  9  9  9  9  5  2 11 11  2 13  4  3  6  2 17 11 13 13  5  3 16  7
  5 13  9  7  2 13  7 16 13  0 13  5  3  2  3  9  9  3  6  7  9  5  9 13
  7  7 13 16 16  9 12  7  5 13  7  3 13  9 13 19]
    train   0% of an epoch | Loss(classif.): 0.3071 | 
    train  27% of an epoch | Loss(classif.): 0.2029 | 
    train  54% of an epoch | Loss(classif.): 0.3070 | 
    train  81% of an epoch | Loss(classif.): 0.0028 | 
TRAIN | Loss(classif.): 0.209 Acc(classif.): 0.982 Macro-F1: 0.978 Micro-F1: 0.982 | 
    valid_class   0% of an epoch | Loss(classif.): 1.9332 | 
    valid_class  81% of an epoch | Loss(classif.): 1.2898 | 


 49%|████▉     | 49/100 [10:29<10:55, 12.84s/it]

VALID_CLASS | Loss(classif.): 1.632 Acc(classif.): 0.736 Macro-F1: 0.652 Micro-F1: 0.736 | 

Epoch 50/100
labels_classification [ 5 19  9  6  0 13 18  9 18  8  9 13  9  7  9 13 13 11 18 13  9  9  9  9
  7  1  9 11  5 19  9 18 13 16 16 11  5 15  3 13 16 13  9 16  7 13  6 16
  7  9  6  0  7  7 13  5 13 13  9  7  8 13  5  9]
preds_classification [ 5 19  9  6  0 13 18  9 18  8  9 13  9  7  9 13 13 11 18 13  9  9  9  9
  7  1  9 11  5 19  9 18 13 16 16 11  5 15  3 13 16 13  9 16  7 13  6 16
  7  9  6  0  7  7 13  5 13 13  9  7  8 13  5  9]
    train   0% of an epoch | Loss(classif.): 0.3036 | 
    train  27% of an epoch | Loss(classif.): 0.3029 | 
    train  54% of an epoch | Loss(classif.): 0.1303 | 
    train  81% of an epoch | Loss(classif.): 0.1728 | 
TRAIN | Loss(classif.): 0.216 Acc(classif.): 0.981 Macro-F1: 0.979 Micro-F1: 0.981 | 
    valid_class   0% of an epoch | Loss(classif.): 2.0465 | 
    valid_class  81% of an epoch | Loss(classif.): 1.4044 | 


 49%|████▉     | 49/100 [10:42<11:09, 13.12s/it]

VALID_CLASS | Loss(classif.): 1.634 Acc(classif.): 0.734 Macro-F1: 0.646 Micro-F1: 0.734 | 
Early stop at epoch 49.
Training complete in 10m 43s
Best val Loss: 1.320658
Best micro f1 score: 0.743502


In [49]:
experiment(batch_size=32, n_epochs=100, lr=1e-3, dim_embedding=256, dim_hidden=256, dropout=0.5, complete=False)

device:  cuda:0
inputs.shape torch.Size([32, 6714]) torch.Size([32, 59])
labels.shape torch.Size([32, 20]) torch.Size([32])
CCNet(
  (embedding): Embedding(6715, 256, padding_idx=6714)
  (encoder): ModuleList(
    (0): ISAB(
      (mab0): MAB(
        (fc_q): Linear(in_features=256, out_features=256, bias=True)
        (fc_k): Linear(in_features=256, out_features=256, bias=True)
        (fc_v): Linear(in_features=256, out_features=256, bias=True)
        (ln0): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ln1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (fc_o): Sequential(
          (0): Linear(in_features=256, out_features=256, bias=True)
          (1): ReLU()
          (2): Linear(in_features=256, out_features=256, bias=True)
        )
        (Dropout): Dropout(p=0.5, inplace=False)
      )
      (mab1): MAB(
        (fc_q): Linear(in_features=256, out_features=256, bias=True)
        (fc_k): Linear(in_features=256, out_features=256, bias=True)
 

  0%|          | 0/100 [00:00<?, ?it/s]


Epoch 1/100
labels_classification [ 9 16  9  5 13 11  9  3  9 16 11 16  3 18  9 14  7 13 11 16 13 13  5 16
  3 13  9  7 13  5  5  5]
preds_classification [ 9 15  8  7  1  2  1 11 18  1  9  7 11 18 11 18  7  7  5  1  1 18 16  2
  1 18 15  1 11  1 12  2]
    train   0% of an epoch | Loss(classif.): 3.6938 | 
    train  13% of an epoch | Loss(classif.): 2.0757 | 
    train  27% of an epoch | Loss(classif.): 2.4877 | 
    train  40% of an epoch | Loss(classif.): 2.0737 | 
    train  54% of an epoch | Loss(classif.): 2.2069 | 
    train  67% of an epoch | Loss(classif.): 1.2786 | 
    train  81% of an epoch | Loss(classif.): 1.5249 | 
    train  95% of an epoch | Loss(classif.): 1.4689 | 
TRAIN | Loss(classif.): 1.922 Acc(classif.): 0.518 Macro-F1: 0.258 Micro-F1: 0.518 | 
    valid_class   0% of an epoch | Loss(classif.): 1.8309 | 
    valid_class  40% of an epoch | Loss(classif.): 2.1761 | 
    valid_class  81% of an epoch | Loss(classif.): 1.6532 | 


  1%|          | 1/100 [00:24<41:02, 24.88s/it]

VALID_CLASS | Loss(classif.): 1.539 Acc(classif.): 0.595 Macro-F1: 0.353 Micro-F1: 0.595 | 

Epoch 2/100
labels_classification [14  7  9 13  9  8  9  9  3  8  3 10 11 13  9  9 13 19 13 16  7 13 16 13
 17  7 13 17  0 16 17  5]
preds_classification [14  7  9 13  9  5  9  9  3  9 11 18  3 13  9  9 13 16 13 16  7 13 16 13
  9  7 13  9 14 16  5  9]
    train   0% of an epoch | Loss(classif.): 1.2422 | 
    train  13% of an epoch | Loss(classif.): 1.0062 | 
    train  27% of an epoch | Loss(classif.): 1.1029 | 
    train  40% of an epoch | Loss(classif.): 1.3855 | 
    train  54% of an epoch | Loss(classif.): 1.1372 | 
    train  67% of an epoch | Loss(classif.): 1.6561 | 
    train  81% of an epoch | Loss(classif.): 1.3394 | 
    train  95% of an epoch | Loss(classif.): 1.4642 | 
TRAIN | Loss(classif.): 1.339 Acc(classif.): 0.642 Macro-F1: 0.422 Micro-F1: 0.642 | 
    valid_class   0% of an epoch | Loss(classif.): 0.8817 | 
    valid_class  40% of an epoch | Loss(classif.): 1.0538 | 
    va

...

VALID_CLASS | Loss(classif.): 1.861 Acc(classif.): 0.726 Macro-F1: 0.637 Micro-F1: 0.726 | 

Epoch 50/100
labels_classification [ 5 19  9  6  0 13 18  9 18  8  9 13  9  7  9 13 13 11 18 13  9  9  9  9
  7  1  9 11  5 19  9 18]
preds_classification [ 5 19  9  6  0 13 18  9 18  8  9 13  9  7  9 13 13 11 18 13  9  9  9  9
  7  1  9 11  5 19  9 18]
    train   0% of an epoch | Loss(classif.): 0.4534 | 
    train  13% of an epoch | Loss(classif.): 0.4040 | 
    train  27% of an epoch | Loss(classif.): 0.5028 | 
    train  40% of an epoch | Loss(classif.): 0.3570 | 
    train  54% of an epoch | Loss(classif.): 0.4030 | 
    train  67% of an epoch | Loss(classif.): 0.4592 | 
    train  81% of an epoch | Loss(classif.): 0.2826 | 
    train  95% of an epoch | Loss(classif.): 0.3618 | 
TRAIN | Loss(classif.): 0.380 Acc(classif.): 0.984 Macro-F1: 0.981 Micro-F1: 0.984 | 
    valid_class   0% of an epoch | Loss(classif.): 1.9360 | 
    valid_class  40% of an epoch | Loss(classif.): 1.1975 | 
    v

 50%|█████     | 50/100 [20:30<20:44, 24.90s/it]

VALID_CLASS | Loss(classif.): 1.883 Acc(classif.): 0.724 Macro-F1: 0.635 Micro-F1: 0.724 | 

Epoch 51/100
labels_classification [ 9  7 11  2 13 16  9  2 18 16 16  9 14  9 13  6 13  3  7 13 13  5  1 13
  3  7 13  3  1 18 13  9]
preds_classification [ 9  7 11  2 13 16  9  2 18 16 16  9 14  9 13  6 13  3  7 13 13  9  1 13
  3  7 13  3  1 18 13  9]
    train   0% of an epoch | Loss(classif.): 0.4631 | 
    train  13% of an epoch | Loss(classif.): 0.3527 | 
    train  27% of an epoch | Loss(classif.): 0.4154 | 
    train  40% of an epoch | Loss(classif.): 0.3529 | 
    train  54% of an epoch | Loss(classif.): 0.4035 | 
    train  67% of an epoch | Loss(classif.): 0.2042 | 
    train  81% of an epoch | Loss(classif.): 0.4033 | 
    train  95% of an epoch | Loss(classif.): 0.2517 | 
TRAIN | Loss(classif.): 0.377 Acc(classif.): 0.984 Macro-F1: 0.981 Micro-F1: 0.984 | 
    valid_class   0% of an epoch | Loss(classif.): 1.1976 | 
    valid_class  40% of an epoch | Loss(classif.): 1.4175 | 
    v

 50%|█████     | 50/100 [20:55<20:55, 25.11s/it]

VALID_CLASS | Loss(classif.): 1.877 Acc(classif.): 0.726 Macro-F1: 0.636 Micro-F1: 0.726 | 
Early stop at epoch 50.
Training complete in 20m 56s
Best val Loss: 1.391723
Best micro f1 score: 0.741335


In [50]:
experiment(batch_size=16, n_epochs=100, lr=1e-3, dim_embedding=256, dim_hidden=256, dropout=0.5, complete=False) # best loss (Macro f1 0.652)

device:  cuda:0
inputs.shape torch.Size([16, 6714]) torch.Size([16, 59])
labels.shape torch.Size([16, 20]) torch.Size([16])
CCNet(
  (embedding): Embedding(6715, 256, padding_idx=6714)
  (encoder): ModuleList(
    (0): ISAB(
      (mab0): MAB(
        (fc_q): Linear(in_features=256, out_features=256, bias=True)
        (fc_k): Linear(in_features=256, out_features=256, bias=True)
        (fc_v): Linear(in_features=256, out_features=256, bias=True)
        (ln0): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ln1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (fc_o): Sequential(
          (0): Linear(in_features=256, out_features=256, bias=True)
          (1): ReLU()
          (2): Linear(in_features=256, out_features=256, bias=True)
        )
        (Dropout): Dropout(p=0.5, inplace=False)
      )
      (mab1): MAB(
        (fc_q): Linear(in_features=256, out_features=256, bias=True)
        (fc_k): Linear(in_features=256, out_features=256, bias=True)
 

  0%|          | 0/100 [00:00<?, ?it/s]


Epoch 1/100
labels_classification [ 9 16  9  5 13 11  9  3  9 16 11 16  3 18  9 14]
preds_classification [ 9 15  2  7  1  2  1 15 12  1  9  7 11 18 11 18]
    train   0% of an epoch | Loss(classif.): 3.6038 | 
    train   6% of an epoch | Loss(classif.): 2.4403 | 
    train  13% of an epoch | Loss(classif.): 2.2969 | 
    train  20% of an epoch | Loss(classif.): 1.4635 | 
    train  27% of an epoch | Loss(classif.): 3.4418 | 
    train  33% of an epoch | Loss(classif.): 1.9399 | 
    train  40% of an epoch | Loss(classif.): 2.1434 | 
    train  47% of an epoch | Loss(classif.): 1.2702 | 
    train  54% of an epoch | Loss(classif.): 1.8784 | 
    train  61% of an epoch | Loss(classif.): 2.1134 | 
    train  67% of an epoch | Loss(classif.): 1.7582 | 
    train  74% of an epoch | Loss(classif.): 1.8381 | 
    train  81% of an epoch | Loss(classif.): 2.5316 | 
    train  88% of an epoch | Loss(classif.): 1.5160 | 
    train  95% of an epoch | Loss(classif.): 1.9882 | 
TRAIN | Loss(classi

  1%|          | 1/100 [00:47<1:18:55, 47.83s/it]

VALID_CLASS | Loss(classif.): 1.677 Acc(classif.): 0.562 Macro-F1: 0.265 Micro-F1: 0.562 | 

Epoch 2/100
labels_classification [14  7  9 13  9  8  9  9  3  8  3 10 11 13  9  9]
preds_classification [ 7  7  9 13  9  5  9  9  3 16  3 18  3 13  9  9]
    train   0% of an epoch | Loss(classif.): 1.2980 | 
    train   6% of an epoch | Loss(classif.): 1.6124 | 
    train  13% of an epoch | Loss(classif.): 1.2586 | 
    train  20% of an epoch | Loss(classif.): 1.9681 | 
    train  27% of an epoch | Loss(classif.): 1.4632 | 
    train  33% of an epoch | Loss(classif.): 1.3097 | 
    train  40% of an epoch | Loss(classif.): 1.2918 | 
    train  47% of an epoch | Loss(classif.): 1.8883 | 
    train  54% of an epoch | Loss(classif.): 1.1721 | 
    train  61% of an epoch | Loss(classif.): 1.8646 | 
    train  67% of an epoch | Loss(classif.): 2.2705 | 
    train  74% of an epoch | Loss(classif.): 1.0639 | 
    train  81% of an epoch | Loss(classif.): 1.4030 | 
    train  88% of an epoch | Loss(cla

...

VALID_CLASS | Loss(classif.): 1.673 Acc(classif.): 0.717 Macro-F1: 0.622 Micro-F1: 0.717 | 

Epoch 54/100
labels_classification [ 9  3  7  5 16  9  9  9  3 16  7  9 13 16 13 16]
preds_classification [ 9  3  7  5 16  9  9  9  3 16  7  9 13 16 13 16]
    train   0% of an epoch | Loss(classif.): 0.7010 | 
    train   6% of an epoch | Loss(classif.): 0.6575 | 
    train  13% of an epoch | Loss(classif.): 0.5566 | 
    train  20% of an epoch | Loss(classif.): 0.5033 | 
    train  27% of an epoch | Loss(classif.): 0.6109 | 
    train  33% of an epoch | Loss(classif.): 0.4847 | 
    train  40% of an epoch | Loss(classif.): 0.6532 | 
    train  47% of an epoch | Loss(classif.): 0.6068 | 
    train  54% of an epoch | Loss(classif.): 0.5052 | 
    train  61% of an epoch | Loss(classif.): 0.6062 | 
    train  67% of an epoch | Loss(classif.): 0.6513 | 
    train  74% of an epoch | Loss(classif.): 0.6843 | 
    train  81% of an epoch | Loss(classif.): 0.5579 | 
    train  88% of an epoch | Loss(cl

 54%|█████▍    | 54/100 [43:09<36:57, 48.21s/it]

VALID_CLASS | Loss(classif.): 1.663 Acc(classif.): 0.719 Macro-F1: 0.624 Micro-F1: 0.719 | 

Epoch 55/100
labels_classification [ 7 12  1  9 12 18 13  5  7 16 18 16  9  5 16  9]
preds_classification [ 7 12  1  9 12 18 13  5  7 16 18 16  9  5 16  9]
    train   0% of an epoch | Loss(classif.): 0.6024 | 
    train   6% of an epoch | Loss(classif.): 0.6587 | 
    train  13% of an epoch | Loss(classif.): 0.5536 | 
    train  20% of an epoch | Loss(classif.): 0.4534 | 
    train  27% of an epoch | Loss(classif.): 0.5893 | 
    train  33% of an epoch | Loss(classif.): 0.5535 | 
    train  40% of an epoch | Loss(classif.): 0.5307 | 
    train  47% of an epoch | Loss(classif.): 0.7060 | 
    train  54% of an epoch | Loss(classif.): 0.5153 | 
    train  61% of an epoch | Loss(classif.): 0.5069 | 
    train  67% of an epoch | Loss(classif.): 0.5525 | 
    train  74% of an epoch | Loss(classif.): 0.6057 | 
    train  81% of an epoch | Loss(classif.): 0.5212 | 
    train  88% of an epoch | Loss(cl

 54%|█████▍    | 54/100 [43:56<37:25, 48.82s/it]

VALID_CLASS | Loss(classif.): 1.662 Acc(classif.): 0.718 Macro-F1: 0.622 Micro-F1: 0.718 | 
Early stop at epoch 54.
Training complete in 43m 56s
Best val Loss: 1.203238
Best micro f1 score: 0.741718


In [ ]:
# Experiment 2. completion only
experiment(batch_size=32, n_epochs=50, lr=1e-2, dim_embedding=128, dropout=0.5, classify=False, exclude_idx=list_single_ingredient_recipe)

In [96]:
# Experiment 3. classification + completion 
experiment(batch_size=64, n_epochs=100, lr=1e-3, dim_embedding=256, dropout=0.5, exclude_idx=list_single_ingredient_recipe)

device:  cuda:0
inputs.shape torch.Size([64, 6714]) torch.Size([64, 59])
labels.shape torch.Size([64, 20]) torch.Size([64])
CCNet(
  (embedding): Embedding(6715, 256, padding_idx=6714)
  (encoder): ModuleList(
    (0): ISAB(
      (mab0): MAB(
        (fc_q): Linear(in_features=128, out_features=128, bias=True)
        (fc_k): Linear(in_features=256, out_features=128, bias=True)
        (fc_v): Linear(in_features=256, out_features=128, bias=True)
        (ln0): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (fc_o): Sequential(
          (0): Linear(in_features=128, out_features=128, bias=True)
          (1): ReLU()
          (2): Linear(in_features=128, out_features=128, bias=True)
        )
        (Dropout): Dropout(p=0.5, inplace=False)
      )
      (mab1): MAB(
        (fc_q): Linear(in_features=256, out_features=128, bias=True)
        (fc_k): Linear(in_features=128, out_features=128, bias=True)
 

  0%|          | 0/100 [00:00<?, ?it/s]


Epoch 1/100
labels_classification [13 19 14 16  9 16  9 12 13 13  5  7 19 19 14 16  9  9  1  7  5 16  5  0
  3  7 19 16  7  9  9  4 10  7  3  5 11  9 15 10  6 13 13 16  5 18  9 13
 13  9  7  3  7 14 13  9  7  5 13 17  3 16  6  3]
preds_classification [15 12  1 12 14  1 16  1 15 14 14 18  1 14  3  1  2 14  8 16  3  3  1  5
 14 14 14 12  3  9  8 15  8 18  3  8  2  1 12 14  3 11 15 12 15  8  0  8
  8 14 14  1  1 12 15  8  8 10  8 18 14  8 14  1]
labels_completion [5249 5216 2693   80 2564 1495 4178 4672 1727 3202 2338 2423 3447 2414
 4844 2258 5884 2122 3597 2356 5338 5908  937 6352 3221 5136 4475  277
 3144 4799 2809  937 2851 3232 4544  317 1592 1308 5648  802 3653 1555
 4617 5454   58  265 1211 4632  167 2564 5536  962 4009 6422 4361  918
  308  585 2945 5609 2414 4454  749 3653]
preds_completion [6294  458 4230 4230 3264 1407  647   25 3110 5825 3545 2767 5790 3740
 4212 3956 1268 2710 3999 3956  822 6632 1253 2951 5274 4212 5670 3085
 5251  458 6555 4109 3177 5719 4230 5274 4061 527

  1%|          | 1/100 [00:20<33:43, 20.44s/it]

VALID_COMPL | Loss(complet.): 7.159 Acc(complet.): 0.064 | 

Epoch 2/100
labels_classification [13 11  5  7 13  7  9  2  7  5 16  5 13 19  3  7 16  9  9 12 13  7  9 15
  3  4  9 13  8 13  9  4  6  9  9 12  2 12 16 13  5 13  7 11 16 13  6 18
  3 14  3 12 13  5 13 13  1 18  5 18  3 13  2  9]
preds_classification [13 16  9  7 13  7  9 14  7  9 16  9  7 18  7 16 16  9 16  3 16  7  9 16
 16 18  9 13 16 13  2  5  9  5  9  3  9 18  9  2  3  2  7  3  9 13  9  7
 18  2  3  3 13 16 13 16 16 16  9 18  3 16  2  9]
labels_completion [ 366  167 4224 5510 4544 2945 4224 5119 5342 5426 4839 1454 6156 2790
 6361 3410 2710 6484 1130 1842 3622 1308 3311  281 4720  579 5648 5536
 4649  513  763  469 3562 4827 5932 1308 1816 2945 3965  962 1007  186
 6187 3274 5377 1390 6344 2118 4672 1137 3978  937  861 1679 6711 1931
  937  962 3146 1780  167 2464 6403 5080]
preds_completion [5536  937 2518 5536  937 5536  937 5536  937  937 1308  937  937 3978
  937 5536  937 5377 5648 3978 2945  937  937 4799  937 2945

...

VALID_COMPL | Loss(complet.): 6.644 Acc(complet.): 0.079 | 

Epoch 83/100
labels_classification [ 0  9  2  6  9 14 13  3  5  2 13  2  5 13  3 13  6 13  9  0  8 12 16  9
  6  6  9 11  0  6 16  7 16 13  9 13 13  3  5  6  9  9 18  9  9  9  9  9
  9  9 13 16  3  7  7  9  2  9 13 18 17  5 11  3]
preds_classification [ 0  9  2  6  9 14 13  3  5  2 13  2  5 13  3 13  6 13  9  0  1 12 16  6
  6  6  9 11  0  6 13  7 16 13  9 13 13 11  5  6  9  9 18  9  9  9  9  9
  9  9 13 16  3  7  7  9  2  9 13 18 17  5 11  3]
labels_completion [3371 3996 4945 4544 3622 2122 4255 5408 4934 1354 4266  937 3152 6126
 1322  962 3097 4361  180 2945 1308 4843  527 1043 4844 4685  210  265
 3078 3152  379 5740 1495 6550 6429 4335 5884 6230 4003 6502 2122 5536
 1308 1816 2338 1772 3205 1476 4260 4395 5326   69 6067 5884 5162 2518
 6610 6599 1495 5459 5882  988 2945 3352]
preds_completion [1308 5377 5377 5377 5377 5536 5536 3978 5774  937 4266  937 1679 1909
  937  937  937  937  937 2945  937 1308 1308  937  937 537

 83%|████████▎ | 83/100 [28:21<05:47, 20.46s/it]

VALID_COMPL | Loss(complet.): 6.645 Acc(complet.): 0.080 | 

Epoch 84/100
labels_classification [ 6 16  3 13 13 13  9  3  9 16 16  1  9 19  9  7 13  0  1  3  9 18 11 13
 15  3 17  1  3  7 11  9 17  9  7  9  5 13 18 16 13 15 13  9  7  9  2  9
  5 13  9 16 16 13  7 16 16 13  5  9  9 18  9 16]
preds_classification [ 6 16  3 13 13  3  9  3  9 16 16 16  9 19  9  7 13  0  1  3  9 18  7 13
 15  3 17  1  3  7 11  9 17  9  7  9  5 13 18 16 13 15 13  9  7  9  2 17
  5 13  5 16 16 13  7 16 16 13  5  9  9 18  9 16]
labels_completion [4885 1308 1661 6512  962 1133 2866 6187 4266  937  937 3891 6352 3447
 5884  916 4361 6276 1111 1112 5882 1790 3147 4361  281   34  937 5368
  167 2634 3978 1476  265 2998 5554 6187  182  937 4216 1679 6138 5970
 6230 5884 5536 3268  167 2173 4649  265 5119 2437 3653 2122  937 3333
 2518 3653 2405 5648 1816 1131 3653 2518]
preds_completion [5377 5648  937 1909  937 1308  937  937  937  937  937  937 2813 1308
 5377 1435 5377 6187  937  937 4799 1790 5536 5536 4799 467

 83%|████████▎ | 83/100 [28:42<05:52, 20.75s/it]

VALID_COMPL | Loss(complet.): 6.646 Acc(complet.): 0.079 | 
Early stop at epoch 83.
Training complete in 28m 43s
Best val Loss: 6.642127


In [ ]:
experiment(batch_size=64, n_epochs=100, lr=1e-3, dim_embedding=256, dropout=0.5, complete=False)

## Dataset for Completion

## Training function for completion

## Experiment for completion